In [1]:
cfg = [
    'DOC_TYPE' : 'Examination',
    'UNIVERSITY' : 'National Taiwan Normal University',
    'EXAMINER': ''
]

email = "mailto:jacky.baltes@ntnu.edu.tw"

import datetime
date = datetime.datetime.now().strftime("%d %B %Y")


In [2]:
import subprocess
import pathlib
import os
import sys
import platform
from importlib import reload 
import logging

logger = logging.getLogger(__name__)
logger.setLevel( logging.DEBUG )

try:
    GIT_CMD
except NameError:
    GIT_CMD = 'git'

try:
    cfg
except NameError:
    cfg = {}
    
class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = pathlib.Path(newPath).expanduser().resolve()

    def __enter__(self):
        self.savedPath = pathlib.Path.cwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

def updateGit( url, dirname, branch,  root ):
        with cd( root ):
            p = pathlib.Path( dirname )
            if ( branch ):
                bs = " --branch " + branch
            else:
                bs = ""
            if not p.is_dir():
                print("cloning {0} from url {1} root {2}".format( dirname, url, root ), 'git command', GIT_CMD)
                    
                cmd = GIT_CMD + " clone " + bs + " " + url + " " + dirname 
                os.system( cmd )
            else:
                logger.info("git directory exists")

            with cd( dirname ):
                logger.info("Executing git pull")
                o = None
                try:
                    o = subprocess.check_output(GIT_CMD + " pull", stderr=subprocess.STDOUT, shell=True)
                except subprocess.CalledProcessError:
                    pass
                if ( o ):
                    logger.info( 'git pull:' + o.decode('utf-8') )

updateGit('https://github.com/cvroberto21/NTNU-Lectures.git', 'NTNU-Lectures', 'exam', '.')

d = str( pathlib.Path( pathlib.Path('.') / 'NTNU-Lectures' ).resolve() )
if d not in sys.path:    
    sys.path.append(  d )
logger.debug('System Path %s', sys.path)

import jblecture

jblecture = reload(jblecture)
node = platform.node()

# %reload_ext jblecture
jblecture.load_ipython_extension( get_ipython(), cfg )

from jblecture import addJBImage, addJBVideo, addJBData, addJBFigure, addJBGraph, addJBAnimation
from jblecture import addJBCharacter
from jblecture import createTable
from jblecture import instTemplate
from jblecture import _a
from jblecture import downloadDir, zipDirectory
from IPython.core.display import display, HTML, Math

doc = cfg['doc']
GDrive = None

def createRevealJSAndDownload():
    logger.info('Create reveal.js and download it')
    doc.createRevealDownload( cfg['REVEAL_DIR'] )
    downloadDir( cfg['ROOT_DIR'] / "{title}_reveal.zip".format( title=title ), "reveal.js", cfg['ROOT_DIR'] )

def finalize():
    cfg['TITLE'] = title
    
    doc.createRevealDownload( cfg['REVEAL_DIR'] )
    
    if jblecture.jbgithub.createGitHub( cfg['TITLE'], cfg['ROOT_DIR']):
        logger.debug("Successful upload of presentation")
        print("You can access the presentation at " + cfg['GITHUB_PAGES_URL'] )
    else:
        logger.warning("Upload of presentation failed")


In [3]:
%%slide --id="start"
%%reveal_rst

{{title}}
====================================================

{{date}}

{{author}}

{{email}} 
    

In [4]:
%%slide
%%reveal_html

<h1>Math Test HTML</h1>

<div class="jb-text">
        
    <p>This now seems to work $0.5=\frac{1}{2}$</p>
    
    <p>Test of brackets and \$ [ a=2\\].</p>
</div>

In [5]:
# Test images for png and svg
pic1 = addJBImage("pic1", 0, 0, url="http://en.ntnu.edu.tw/images/index-banner02.jpg" )

In [6]:
# Must add viewbox and width and height attributes
pic2Data = """
<svg viewBox="0 0 777 480" xmlns="http://www.w3.org/2000/svg" xmlns:se="http://svg-edit.googlecode.com" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:cc="http://creativecommons.org/ns#" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:inkscape="http://www.inkscape.org/namespaces/inkscape" width="777">                                    <title>my vector image</title>                                    <!-- Created with Vector Paint - http://www.vectorpaint.yaks.com/ https://chrome.google.com/webstore/detail/hnbpdiengicdefcjecjbnjnoifekhgdo -->                                    <rect id="backgroundrect" width="100%" height="100%" x="0" y="0" fill="#FFFFFF" stroke="none"/>                                <g class="currentLayer" style=""><title>Layer 1</title><rect fill="#4a90d6" stroke="#222222" stroke-width="2" stroke-linejoin="round" stroke-dashoffset="" fill-rule="nonzero" id="svg_1" x="90" y="98" width="147" height="126" style="color: rgb(0, 0, 0);"/><path fill="#4a90d6" fill-opacity="1" stroke="#222222" stroke-opacity="1" stroke-width="2" stroke-dasharray="none" stroke-linejoin="round" stroke-linecap="butt" stroke-dashoffset="" fill-rule="nonzero" opacity="1" marker-start="" marker-mid="" marker-end="" d="M211.5213985805299,276.6071114920984L444.55711187410054,175.71425477879006" id="svg_3"/><path fill="#4a90d6" fill-opacity="1" stroke="#222222" stroke-opacity="1" stroke-width="2" stroke-dasharray="none" stroke-linejoin="round" stroke-linecap="butt" stroke-dashoffset="" fill-rule="nonzero" opacity="1" marker-start="" marker-mid="" marker-end="" id="svg_6" d="M 315.98568385006155 123.0356835744963 C 344.557112299848 174.82139763973424 293.6642553736659 205.17854036763237 292.85714285714283 204.4642857142857 C 293.6642553736659 205.17854036763237 212.41425571958573 250.71425445947952 211.60714285714283 249.99999999999997 C 212.41425571958573 250.71425445947952 97.23568478138415 299.82139710754996 97.23568478138415 301.6071113856616 C 97.23568478138415 303.39282566377324 159.73568451529198 381.9642539006859 158.92857142857142 381.24999999999994 C 159.73568451529198 381.9642539006859 276.6999697316052 389.10711101313257 275.8928571428571 388.3928571428571 C 276.6999697316052 389.10711101313257 411.52139772903496 350.714254033732 410.71428571428567 349.99999999999994 C 411.52139772903496 350.714254033732 497.2356830783943 271.2499686577635 497.2356830783943 271.2499686577635 C 497.2356830783943 271.2499686577635 521.3428258329017 192.67854042085077 520.4499686938458 190.89282614273912 C 519.5571115547899 189.10711186462748 450.80711184749134 115.8928264620497 442.7713975959889 114.99996932299388 C 434.73568334448646 114.10711218393806 345.44996943890385 91.78568370754238 339.19996946551305 89.10711229037491" style="color: rgb(0, 0, 0);"/><foreignObject fill="#41e841" stroke="#222222" stroke-width="2" stroke-linejoin="round" stroke-dashoffset="" fill-rule="nonzero" font-size="50" font-family="Georgia, serif" letter-spacing="0" word-spacing="0" marker-start="" marker-mid="" marker-end="" id="svg_2" x="540" y="298" width="198" height="96" style="color: rgb(0, 0, 0);" class=""><p xmlns="http://www.w3.org/1999/xhtml" style="border: none;outline: none;font-size: inherit;line-height: 1em;padding:0;margin:0;">Hello<div>World</div></p></foreignObject></g></svg>
"""

pic2 = addJBImage( "pic2", 0, 0, data = pic2Data.encode('utf-8'), suffix="svg")


In [7]:
%%slide
%%reveal_html

<h1>NTNU</h1>

<p>PNG, JPG and SVG Images are supported.</p>

<table>
<tr>
<td>
<div class="jb-center" style="width:100%;">
{{ pic1(cls="", style="") }}
</div>
</td>
<td>
<div class="jb-center" style="width:100%;">
{{ pic2() }}
</div>
</td>
</tr>
</table>

,"my vector image <!-- Created with Vector Paint - http://www.vectorpaint.yaks.com/ https://chrome.google.com/webstore/detail/hnbpdiengicdefcjecjbnjnoifekhgdo --> Layer 1<path fill=""#4a90d6"" fill-opacity=""1"" stroke=""#222222"" stroke-opacity=""1"" stroke-width=""2"" stroke-dasharray=""none"" stroke-linejoin=""round"" stroke-linecap=""butt"" stroke-dashoffset="""" fill-rule=""nonzero"" opacity=""1"" marker-start="""" marker-mid="""" marker-end="""" id=""svg_6"" d=""M 315.98568385006155 123.0356835744963 C 344.557112299848 174.82139763973424 293.6642553736659 205.17854036763237 292.85714285714283 204.4642857142857 C 293.6642553736659 205.17854036763237 212.41425571958573 250.71425445947952 211.60714285714283 249.99999999999997 C 212.41425571958573 250.71425445947952 97.23568478138415 299.82139710754996 97.23568478138415 301.6071113856616 C 97.23568478138415 303.39282566377324 159.73568451529198 381.9642539006859 158.92857142857142 381.24999999999994 C 159.73568451529198 381.9642539006859 276.6999697316052 389.10711101313257 275.8928571428571 388.3928571428571 C 276.6999697316052 389.10711101313257 411.52139772903496 350.714254033732 410.71428571428567 349.99999999999994 C 411.52139772903496 350.714254033732 497.2356830783943 271.2499686577635 497.2356830783943 271.2499686577635 C 497.2356830783943 271.2499686577635 521.3428258329017 192.67854042085077 520.4499686938458 190.89282614273912 C 519.5571115547899 189.10711186462748 450.80711184749134 115.8928264620497 442.7713975959889 114.99996932299388 C 434.73568334448646 114.10711218393806 345.44996943890385 91.78568370754238 339.19996946551305 89.10711229037491"" style=""color: rgb(0, 0, 0);""/>HelloWorld"


In [8]:
vid1 = addJBVideo( "vid1", 0, 0, url = "https://www.youtube.com/embed/EvJXubQjFWQ", suffix="mp4" )

[youtube] EvJXubQjFWQ: Downloading webpage
[download] Destination: /data/NTNU-Lectures/BuildDir/reveal.js/assets/videos/vid1.mp4
[download] 100% of 9.72MiB in 00:0290MiB/s ETA 00:003


In [9]:
%%slide
%%reveal_html

<h1>Videos</h1>

<table>
<tr>
<td style="width:50%">
{{ cfg['ASSETS']['vid1']() }}
</td>
</tr>
</table>

""


In [10]:
%%slide --echo --style "font-size:0.6em"
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import math
import random

pts = np.random.random((2,10))
pts.sort(axis=1)
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(1,1,1)

ax.plot(pts[0,:], pts[1,:], "-", linewidth=3)

A = np.vstack( ( pts[0,:], np.ones( len( pts[0]) ) ) ).T
b = pts[1,:]


#print(A)
theta = np.linalg.solve( A.T.dot(A), A.T.dot(b) )
m = np.array( [ [ 0, 1 ], [1, 1] ] )
#print(theta_
ax.plot( [ 0, 1 ],  m.dot( theta ) )

svg1 = addJBFigure("svg1", 0, 0, fig );

In [11]:
%%slide
%%reveal_html

<h1> Least Squared Error (LSE) Approximation</h1>

<div style="width:100%;display:block;text-align:center">
{{ svg1() }}
</div>

In [12]:
%%slide --echo --style="font-size:0.6em"
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
import math
import random

pts = np.random.random((10))
#pts = np.arange(0,10, dtype=float)
np.sort( pts )

model = np.random.normal(0,2, 2)
#print('Model', model.shape, model)

xs = np.vstack( ( pts, np.ones(len(pts) ) ) ).T
ys = xs @ model + np.random.normal(0, 0.05, len(xs))

#print(xs.shape, 'Xs', xs, ys.shape, 'Ys', ys)

theta = np.random.normal(0,2, 2)
alpha = 0.1

err = ( xs @ theta - ys ) 
MAX_ERR = np.sum(err ** 2)

fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

line = None

errLine = None

errYs = []

NFRAMES = 50

In [13]:
%%slide --echo --style="font-size:0.6em"

def init():
  global ax, line, ax2, errLine

  ax.scatter( xs[:,0], ys, 20 )

  line, = ax.plot( [], [], 'b-', linewidth=2 )
  errLine, = ax2.plot( [], [], 'b-', linewidth=2 )
  ax2.set_xlim( ( 0,NFRAMES ) )
  ax2.set_ylim( ( 0, MAX_ERR ) )

In [14]:
%%slide --echo --style="font-size:0.6em"

def anim( frame ):
  global theta, xs, ys, errYs

  #print('anim', frame )
  artists = []

  err = ( xs @ theta - ys ) 

  e = np.sum(err ** 2)
  #print('Err', e, 'Model', model, 'Theta', theta )
  errYs.append( e )

  #print(xs.shape)
  grad = np.sum(- xs * err.reshape((len(err),1)), axis=0)
  #print('Gradient', grad)
  
  theta = theta + alpha * grad

  line.set_xdata( xs[:,0] )
  line.set_ydata( xs.dot( theta ) )

  errLine.set_xdata( np.arange( len( errYs ) ) )
  errLine.set_ydata( errYs )

  return line, errLine


In [15]:
%%slide --echo --style="font-size:0.6em"

from matplotlib import animation, rc

rc('animation', html='jshtml')

ani = animation.FuncAnimation(fig,anim,init_func=init, interval = 100, frames = NFRAMES )
Writer = animation.writers['ffmpeg']
writer = Writer()

ani.save( cfg['REVEAL_VIDEOS_DIR'] / "ani.mp4", writer=writer)



ani1 = addJBVideo( "ani1", 0, 0, localFileStem = str( cfg['REVEAL_VIDEOS_DIR'] / "ani.mp4" ), suffix="mp4" )
#ani

In [16]:
%%slide
%%reveal_html

<h1>Gradient Descent</h1>

{{ ani1() }}


FileNotFoundError: [Errno 2] No such file or directory: '/data/NTNU-Lectures/BuildDir/reveal.js/assets/videos/ani.mp4.mp4'

In [17]:
finalize()

Github Token: 


In [24]:
import logging

logging.debug("Test")

DEBUG:root:Test


In [ ]:
! cat BuildDir/Test-Implementation/index.html

<!doctype html>
<html>
    <head>
        <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">

        <title>Test Implementation</title>

        <link rel="stylesheet" href="css/reveal.css">
        <link rel="stylesheet" href="css/theme/ntnuerc.css">

        <!-- Theme used for syntax highlighting of code -->
        <link rel="stylesheet" href="lib/css/zenburn.css">
        <script src="https://daybrush.com/scenejs/release/latest/dist/scene.min.js"></script>
		
        <!-- Printing and PDF exports -->
        <script>
            var link = document.createElement( 'link' );
            link.rel = 'stylesheet';
            link.type = 'text/css';
            link.href = window.location.search.match( /print-pdf/gi ) ? 'css/print/pdf.css' : 'css/print/paper.css';
            document.getElementsByTagName( 'head' )[0].appendChild( link );
        </script>
        <script src="js/ntnu.js"></script>